Name: Erik Alexander Sandvik <br />
Course: STK-IN4300 <br />
Assignment number: 2 <br />

In [16]:
%matplotlib inline

# Import required packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Problem 1. Regression

### 1.

In [17]:
# Read data from file

df = pd.read_csv("qsar_aquatic_toxicity.csv", sep=";")


# Label the columns

names = ['TPSA', 'SAacc', 'H050', 'MLOGP', 'RDCHI', 'GATS1p', 'nN', 'C040', 'LC50']
df.columns = names


# Split dataframe by features and response variable

X = df.iloc[:, :-1]
y = df.iloc[:, -1]


# Split the data into a test and training set

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)


# Ordinary Least Squares

from sklearn import linear_model
reg = linear_model.LinearRegression()

reg.fit(X_train, y_train)


# Calculate the predicted response using the training set and test set as inputs

y_predict_train = reg.predict(X_train)
y_predict_test = reg.predict(X_test)


# Calculate the training and test error

from sklearn.metrics import mean_squared_error as MSE

print('Training error: {:.2f}'.format(MSE(y_train, y_predict_train)))
print('Test error: {:.2f}'.format(MSE(y_test, y_predict_test)))

Training error: 1.20
Test error: 1.92


In [18]:
# Calculate the significance level of each coefficient in the linear model
# scikit-learn doesn't have this implemented

import statsmodels.api as sm
from statsmodels.tools import add_constant

mod = sm.OLS(y_train, add_constant(X_train))
fii = mod.fit()
p_values = fii.summary2().tables[1]['P>|t|']

print("P-values:\n")
print(p_values)

P-values:

const     6.718007e-19
TPSA      4.488061e-17
SAacc     5.134547e-10
H050      4.744807e-01
MLOGP     4.216208e-12
RDCHI     5.209703e-03
GATS1p    6.349095e-04
nN        5.909546e-03
C040      7.280372e-01
Name: P>|t|, dtype: float64


In [19]:
# Same thing as above, but with dichotomization of the count variables


# Subscript d for dichotomization

X_d = X.copy()

count_variables = ['H050', 'nN', 'C040']

for cnt_var in count_variables:
    X_d.loc[X[cnt_var] > 0, cnt_var] = 1
    
X_train_d, X_test_d, y_train, y_test = train_test_split(X_d, y, test_size=0.33, random_state=1)

reg_d = linear_model.LinearRegression()

reg_d.fit(X_train_d, y_train)

y_predict_train_d = reg_d.predict(X_train_d)
y_predict_test_d = reg_d.predict(X_test_d)

print('Training error: {:.2f}'.format(MSE(y_train, y_predict_train_d)))
print('Test error: {:.2f}'.format(MSE(y_test, y_predict_test_d)))

mod = sm.OLS(y_train, add_constant(X_train_d))
fii = mod.fit()
p_values = fii.summary2().tables[1]['P>|t|']

print('\n')
print('P-values:\n')
print(p_values)

Training error: 1.22
Test error: 2.02


P-values:

const     5.831407e-20
TPSA      4.879928e-13
SAacc     1.190513e-08
H050      1.693085e-01
MLOGP     1.701730e-12
RDCHI     1.398751e-02
GATS1p    4.313739e-04
nN        6.238373e-01
C040      3.955416e-01
Name: P>|t|, dtype: float64


With dichotomization the test error increases and the p-values of the coefficients increases by orders of magnitude, with the exception of the variable 'C040' where the p-value of the corresponding coefficient is reduced by a factor of ~2.

### 2.

In [20]:
# Repeat 200 times

av_test_error    = 0
av_test_error_d  = 0

av_test_error2   = 0
av_test_error2_d = 0

n = 200

for i in range(n):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=i)
    X_train_d, X_test_d, y_train, y_test = train_test_split(X_d, y, test_size=0.33, random_state=i)
    
    reg.fit(X_train, y_train)
    reg_d.fit(X_train_d, y_train)
    
    y_predict = reg.predict(X_test)
    y_predict_d = reg_d.predict(X_test_d)
    
    av_test_error += MSE(y_test, y_predict)
    av_test_error_d += MSE(y_test, y_predict_d)
    
    av_test_error2 += MSE(y_test, y_predict)**2
    av_test_error2_d += MSE(y_test, y_predict_d)**2
    
av_test_error    /= n
av_test_error_d  /= n
av_test_error2   /= n
av_test_error2_d /= n

std = np.sqrt(av_test_error2 - av_test_error**2)
std_d = np.sqrt(av_test_error2_d - av_test_error_d**2)


print("Average test error: {:.2f}, Standard deviation: {:.2f}".format(av_test_error, std))
print("Average test error dichotomized: {:.2f}, Standard deviation: {:.2f}".format(av_test_error_d, std_d))

Average test error: 1.48, Standard deviation: 0.17
Average test error dichotomized: 1.53, Standard deviation: 0.17


The average test errors are lower than what was obtained before (1.92 and 2.02), but that was just for one particular way of splitting the data for training and testing, where I suppose I just got very unlucky (see the standard deviation). 

My best guess for why dichotomization leads to a worse result is that dichotomization makes the linear model biased. But the variance of the prediction is about the same with or without dichotomization. According to the bias-variance decomposition, the expected prediction error is increased with dichotomization.

### 3.

In [21]:
# The first training/test split again

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
reg.fit(X_train, y_train)

y_predict_test = reg.predict(X_test)

error = MSE(y_test, y_predict_test)

coeffs = reg.coef_

print("Error: {:.2f}".format(error))
print("Coefficients: ", np.around(coeffs, decimals=2))

Error: 1.92
Coefficients:  [ 0.03 -0.02  0.05  0.52  0.43 -0.62 -0.15 -0.04]


In [22]:
# Automated Stepwise forward and backward selection in python
# See https://github.com/talhahascelik/python_stepwiseSelection for details

import stepwiseSelection as ss

# Backward selection with AIC
final_vars, _ = ss.backwardSelection(X_train, y_train, model_type ="linear", elimination_criteria = "aic")


Character Variables (Dummies Generated, First Dummies Dropped): []
Eliminated : C040
Eliminated : H050
                            OLS Regression Results                            
Dep. Variable:                   LC50   R-squared:                       0.550
Model:                            OLS   Adj. R-squared:                  0.543
Method:                 Least Squares   F-statistic:                     72.95
Date:                Mon, 18 Oct 2021   Prob (F-statistic):           4.03e-59
Time:                        15:49:15   Log-Likelihood:                -552.21
No. Observations:                 365   AIC:                             1118.
Df Residuals:                     358   BIC:                             1146.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------

Please excuse all the junk.

We see that the features 'C040' and 'H050' were eliminated. 

In [23]:
X_fs_aic = X.copy()

X_fs_aic.drop('C040', axis=1, inplace=True)
X_fs_aic.drop('H050', axis=1, inplace=True)

X_train_fs_aic, X_test_fs_aic, y_train_fs_aic, y_test_fs_aic = train_test_split(X_fs_aic, y, test_size=0.33, random_state=1)

reg.fit(X_train_fs_aic, y_train_fs_aic)

y_predict_test = reg.predict(X_test_fs_aic)

error = MSE(y_test, y_predict_test)

coeffs = reg.coef_

print("Error: {:.2f}".format(error))
print("Coefficients: ", np.around(coeffs, decimals=2))

Error: 1.92
Coefficients:  [ 0.03 -0.01  0.5   0.43 -0.66 -0.14]


In [24]:
# Backward selection with BIC
final_vars, _ = ss.backwardSelection(X_train, y_train, model_type ="linear", elimination_criteria = "bic")

Character Variables (Dummies Generated, First Dummies Dropped): []
Eliminated : C040
Eliminated : H050
                            OLS Regression Results                            
Dep. Variable:                   LC50   R-squared:                       0.550
Model:                            OLS   Adj. R-squared:                  0.543
Method:                 Least Squares   F-statistic:                     72.95
Date:                Mon, 18 Oct 2021   Prob (F-statistic):           4.03e-59
Time:                        15:49:15   Log-Likelihood:                -552.21
No. Observations:                 365   AIC:                             1118.
Df Residuals:                     358   BIC:                             1146.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------

The same couple of variables were eliminated, so the error and coefficients are the same with backward selection for both AIC and BIC.

In [26]:
# Forward selection with AIC
final_vars, _ = ss.forwardSelection(X_train, y_train, model_type ="linear", elimination_criteria = "aic")

Character Variables (Dummies Generated, First Dummies Dropped): []
Entered : MLOGP 	AIC : 1237.49891950763
Entered : TPSA 	AIC : 1179.6284568355122
Entered : SAacc 	AIC : 1134.7105633054348
Entered : GATS1p 	AIC : 1128.0436044458704
Entered : RDCHI 	AIC : 1123.7030733885122
Entered : nN 	AIC : 1118.4177729055477
Break : Significance Level
                            OLS Regression Results                            
Dep. Variable:                   LC50   R-squared:                       0.550
Model:                            OLS   Adj. R-squared:                  0.543
Method:                 Least Squares   F-statistic:                     72.95
Date:                Mon, 18 Oct 2021   Prob (F-statistic):           4.03e-59
Time:                        15:54:51   Log-Likelihood:                -552.21
No. Observations:                 365   AIC:                             1118.
Df Residuals:                     358   BIC:                             1146.
Df Model:                  

H050 and C040 were eliminated yet again.

In [27]:
# Forward selection with BIC
final_vars, _ = ss.forwardSelection(X_train, y_train, model_type ="linear", elimination_criteria = "bic")

Character Variables (Dummies Generated, First Dummies Dropped): []
Entered : MLOGP 	BIC : 1245.298714214795
Entered : TPSA 	BIC : 1191.3281488962598
Entered : SAacc 	BIC : 1150.3101527197648
Entered : GATS1p 	BIC : 1147.5430912137829
Entered : RDCHI 	BIC : 1147.102457510007
Entered : nN 	BIC : 1145.717054380625
Break : Significance Level
                            OLS Regression Results                            
Dep. Variable:                   LC50   R-squared:                       0.550
Model:                            OLS   Adj. R-squared:                  0.543
Method:                 Least Squares   F-statistic:                     72.95
Date:                Mon, 18 Oct 2021   Prob (F-statistic):           4.03e-59
Time:                        15:55:42   Log-Likelihood:                -552.21
No. Observations:                 365   AIC:                             1118.
Df Residuals:                     358   BIC:                             1146.
Df Model:                   

So both backward and forward selection, with AIC or with BIC, leads to the exclusion of the variables 'H050' and 'C040', which are the third and the last feature respectively. With or without exclusion, the prediction error is about the same. <br />

For your convenience (so you don't have to scroll through all the junk), the coefficients in the linear model without exclusion are <br />

[ 0.03 -0.02  0.05  0.52  0.43 -0.62 -0.15 -0.04] <br />

With exclusion, the coefficients in the linear model are <br />

[ 0.03 -0.01  0.5   0.43 -0.66 -0.14] <br />

The coefficients of the features that are kept have nearly unchanged values, only differing at most in the second decimal place.